In [1]:
!pip install google-cloud-aiplatform --upgrade --user

In [3]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import requests
import pickle
import csv
import seaborn as sns
from IPython.display import display, HTML
from IPython.display import Markdown, display
from sentence_transformers import SentenceTransformer
from vertexai.preview.language_models import (ChatModel, InputOutputTextPair,
                                              TextEmbeddingModel,
                                              TextGenerationModel)

chat_model = ChatModel.from_pretrained("chat-bison@001")
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [50]:
print("Conversation with a friend \n")

chat = chat_model.start_chat()

user_input1 = input("Me: ")
ai_response1= chat.send_message(user_input1)
print("Friend: ", ai_response1)
user_input2 = input("Me: ")
ai_response2= chat.send_message(user_input2)
print("Friend: ", ai_response2)
print("\n Generating my gif suggestion...")

prompt = f"""
Me: {user_input1} 

Friend: {ai_response1}

Me: {user_input2}

Friend: {ai_response2}

Generate a response to Friend from Me in one sentence. 
"""

response = generation_model.predict(prompt=prompt).text

print("\n My suggested response:", response)

f = open ("embedded_gif.pkl", "rb")
emb_gif = pickle.load(f)

retriever = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
def embed(response):
    emb_response = retriever.encode(response).tolist()
    list_cos_sim = []
    for gif in emb_gif:
        cos_sim = abs(np.dot (gif['emb'],emb_response))
        list_cos_sim.append(cos_sim)
    list_cos_sim = np.array(list_cos_sim)
    max_idx = np.argmax(list_cos_sim)
    
    return emb_gif[max_idx]

selected_gif = embed(response)


Conversation with a friend 



Me:  I am so screwed for exams


Friend:  I'm sorry to hear that. What can I do to help?


Me:  Well what subject do you have for tmrw


Friend:  I have math tomorrow.

 Generating my gif suggestion...

 My suggested response: I have math tomorrow too. I'm not sure if I'm ready for it.


In [51]:


HTML(f'''<div style="display: flex; justify-content: row;">
    <img src="{selected_gif['url']}">
</div>''')


In [52]:
print("Gif caption: ", selected_gif['description'])

Gif caption:  a man with glasses is writing down equations for his students until they left afterwards.
